In [1]:
# model training using tensorflow
import numpy as np
import os
import cv2
from matplotlib import pyplot

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
'''
Configuration Code
'''
def get_n_cores():
    """The NSLOTS variable, If NSLOTS is not defined throw an exception."""
    nslots = os.getenv("NSLOTS")
    if nslots is not None:
        return int(nslots)
    raise ValueError("Environment variable NSLOTS is not defined.")


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

NUM_GPUS = len(tf.config.experimental.list_physical_devices("GPU"))
print("Num GPUs Available: ", NUM_GPUS)
if NUM_GPUS > 0:
    print(os.getenv("CUDA_VISIBLE_DEVICES"))

tf.config.set_soft_device_placement(True)
tf.keras.backend.set_floatx("float32")
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(get_n_cores())

Num GPUs Available:  1
0


In [4]:
next(os.walk("models"))[1]

['model_tf_100']

In [5]:
model = keras.models.load_model("models/model_tf_100")

In [27]:
'''
Try the test set
'''

TEST_DIRECTORY = os.path.join(os.getcwd(), 'backup_data', 'formatted_data')

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIRECTORY,
    seed=123,
    image_size=(224,224),
    shuffle=False
)

Found 253 files belonging to 2 classes.


In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 64)       

In [28]:
model.evaluate(test_ds, return_dict=True)

8/8 [==============================] - 0s 54ms/step - loss: 0.2256 - accuracy: 0.3913


{'loss': 0.22558605670928955, 'accuracy': 0.3913043439388275}

In [29]:
predictions = np.argmax(model.predict(test_ds), axis=1)
actual = np.concatenate([y for x, y in test_ds], axis=0)

In [33]:
print("positives:", np.sum(actual),"negatives:", len(actual)-np.sum(actual))

positives: 155 negatives: 98


In [34]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(actual, predictions, labels=[0,1])

array([[ 95,   3],
       [  4, 151]])

In [37]:
from sklearn.metrics import classification_report

report = classification_report(actual, predictions)

In [38]:
print(report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96        98
           1       0.98      0.97      0.98       155

    accuracy                           0.97       253
   macro avg       0.97      0.97      0.97       253
weighted avg       0.97      0.97      0.97       253

